> Code for translating *Atom*'s fields into columns' names

In [ ]:
import pathlib

import pandas as pd
import numpy as np
import ruamel.yaml

Names mapping is expected in the file below (not available in this repository).

In [ ]:
excel_place = pathlib.Path.cwd() / 'PLACE.xlsx'
assert excel_place.exists()
excel_place

PosixPath('/home/manu/dlsproc/make/PLACE.xlsx')

In [ ]:
df = pd.read_excel(excel_place, sheet_name='Agregados sin menores', skiprows=5, usecols='A:I')
df.head()

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Categoría,Dato
0,id,NaN,NaN,NaN,NaN,NaN,NaN,NaN,id
1,summary,NaN,NaN,NaN,NaN,NaN,NaN,NaN,summary
2,title,NaN,NaN,NaN,NaN,NaN,NaN,NaN,title
3,updated,NaN,NaN,NaN,NaN,NaN,NaN,NaN,updated
4,ContractFolderStatus,ContractFolderID,NaN,NaN,NaN,NaN,NaN,Datos Generales del Expediente,Número de Expediente


There are columns that are mapped to the same (*Dato*) name.

In [ ]:
df.iloc[50:]

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Categoría,Dato
50,ContractFolderStatus,LegalDocumentReference,Attachment,ExternalReference,URI,NaN,NaN,Datos Generales del Expediente,Pliego de cláusulas administrativas (URI)
51,ContractFolderStatus,TechnicalDocumentReference,ID,NaN,NaN,NaN,NaN,Datos Generales del Expediente,Pliego de Prescripciones técnicas
52,ContractFolderStatus,TechnicalDocumentReference,Attachment,ExternalReference,URI,NaN,NaN,Datos Generales del Expediente,Pliego de Prescripciones técnicas (URI)
53,ContractFolderStatus,AdditionalDocumentReference,ID,NaN,NaN,NaN,NaN,Datos Generales del Expediente,Anexos a los pliegos
54,ContractFolderStatus,AdditionalDocumentReference,Attachment,ExternalReference,URI,NaN,NaN,Datos Generales del Expediente,Anexos a los pliegos (URI)


Maximum number of levels is inferred

In [ ]:
max_levels = df.shape[1] - 2
max_levels

7

In [ ]:
mapping = {}
for t in df.itertuples(index=False):
    
    # only if there is a "to" for this path in the *Atom*...
    if t[-1] != '???':
        
        if t[-1] in mapping:
            
            print(f'overwriting {t[-1]} -> {mapping[t[-1]]}\nwith {t[:max_levels]}')
        
        # ...the mapping is saved
        mapping[t[-1]] = t[:max_levels]

First few pairs *key: value*

In [ ]:
{k: mapping[k] for k in list(mapping)[:5]}

{'id': ('id', nan, nan, nan, nan, nan, nan),
 'summary': ('summary', nan, nan, nan, nan, nan, nan),
 'title': ('title', nan, nan, nan, nan, nan, nan),
 'updated': ('updated', nan, nan, nan, nan, nan, nan),
 'Número de Expediente': ('ContractFolderStatus',
  'ContractFolderID',
  nan,
  nan,
  nan,
  nan,
  nan)}

In [ ]:
type(mapping['id'][-1])

float

One can manually add some correspondences

In [ ]:
new_path = ('ContractFolderStatus', 'TenderingProcess', 'TenderSubmissionDeadlinePeriod')
mapping['Presentación de Oferta'] = new_path + tuple([np.NAN] * (max_levels - len(new_path)))

In [ ]:
output_file = pathlib.Path('PLACE.yaml')

In [ ]:
yaml = ruamel.yaml.YAML()
yaml.indent(sequence=4, offset=2)
# yaml.default_flow_style = True

with open(output_file, 'w') as f:
    yaml.dump(mapping, f)